In [1]:
from rank_bm25 import BM25Okapi
import sys
sys.path.insert(0, '/home/jianx/search-exposure/forward_ranker/')
from load_data import obj_reader

In [22]:
COLLECTION_PATH = "/home/jianx/data/collection.tsv"
corpus = []
id_mapping = {}
with open(COLLECTION_PATH, "r") as f:
    line = f.readline()
    while line:
        corpus.append(line.split("\t")[1])
        id_mapping[line.split("\t")[1]] = int(line.split("\t")[0])
        line = f.readline()
tokenized_corpus = [doc.split(" ") for doc in corpus]

In [3]:
bm25 = BM25Okapi(tokenized_corpus)

In [13]:
initial_ranking = obj_reader("/home/jianx/data/initial_ranking.dict")

In [36]:
QUERY_TEST_PATH = "/home/jianx/data/msmarco-test2019-queries.tsv"
query_test = {}
with open(QUERY_TEST_PATH,"r") as f:
    line = f.readline()
    while line:
        query_test[int(line.split("\t")[0])] = line.split("\t")[1]
        line = f.readline()
print(len(query_test))

200


In [39]:
rating_dict = obj_reader("/home/jianx/data/rel_scores.dict")
print(rating_dict.keys())

dict_keys([19335, 47923, 87181, 87452, 104861, 130510, 131843, 146187, 148538, 156493, 168216, 182539, 183378, 207786, 264014, 359349, 405717, 443396, 451602, 489204, 490595, 527433, 573724, 833860, 855410, 915593, 962179, 1037798, 1063750, 1103812, 1106007, 1110199, 1112341, 1113437, 1114646, 1114819, 1115776, 1117099, 1121402, 1121709, 1124210, 1129237, 1133167])


In [41]:
result_dict = {}
for qid, query in query_test.items():
    if qid not in rating_dict.keys():
        continue
    scores = bm25.get_scores(query.split(" "))
    score_dict = {}
    count = 0
    for i, score in enumerate(scores):
        count += 1
        score_dict[i] = score
    result_list = [k for k, v in sorted(score_dict.items(), key=lambda item: item[1])]
    result_list.append(result_list[:1000])
    result_dict[qid] = result_list

In [ ]:
from test import get_ndcg_precision_rr
import numpy as np
rating_dict = obj_reader("/home/jianx/data/rel_scores.dict")
rank = 10

qids = list(result_dict.keys())
result_ndcg = []
result_prec = []
result_rr = []
for qid in qids:
    if qid in rating_dict:
        ndcg, prec, rr = get_ndcg_precision_rr(rating_dict[qid], result_dict[qid], rank)
        result_ndcg.append(ndcg)
        result_prec.append(prec)
        result_rr.append(rr)
avg_ndcg = np.nanmean(result_ndcg)
avg_prec = np.nanmean(result_prec)
avg_rr = np.nanmean(result_rr)
print(avg_ndcg, avg_prec, avg_rr)
